# Вычислительный практикум
# Задание №2
### Итерационные методы (простой итерации, Зейделя, верхней релаксации) решения СЛАУ. 
## Ковальчуков Александр
### 321 группа
### Вариант №6

In [228]:
import numpy as np

# Параметры задачи

In [229]:
A = np.array([[ 9.016024, 1.082197, -2.783575],
              [ 1.082197, 6.846595,  0.647647],
              [-2.783575, 0.647647,  5.432541]])


b = np.array([[-1.340873],[4.179164],[5.478007]])

n, m  = A.shape

# Схема Гаусса

In [230]:
def solve_Gauss(A, b):
    A = np.hstack((A, b))
    x = np.zeros((3, 1))

    # Прямой ход
    for k in range(n):
        if abs(A[k, k]) < epsilon:
            raise ZeroDivisionError
        A[k, :] /= A[k, k]
        for i in range(k + 1, n):
            A[i, :] -= A[k, :] * A[i, k]

    # Обратный ход
    for i in range(0, n)[::-1]:
        x[i] = A[i, n] - np.dot(A[i, :-1], x)

    return x

In [231]:
x_g = solve_Gauss(A.copy(), b.copy())
x_g

array([[0.10000016],
       [0.49999994],
       [1.00000009]])

# Метод простой итерации

In [232]:
D = A * np.eye(n)
H_D = np.eye(n) - np.dot(np.linalg.inv(D), A)
g_D = np.dot(np.linalg.inv(D), b)

In [233]:
x0 = np.zeros((n, 1))
x1 = x0

for i in range(10):
  x0 = x1
  x1 = np.dot(H_D, x0) + g_D

In [234]:
H_norm = np.linalg.norm(H_D, ord=np.inf)

### Апостриорная оценка погрешности (по норме $||\; . ||_\infty$)

In [235]:
apost = H_norm / (1 - H_norm) * np.linalg.norm(x0 - x1, ord=np.inf)
apost

0.0004462394087822607

### Фактическая погрешность

In [236]:
fact_it = np.linalg.norm(x - x_g, ord=np.inf)
fact_it

0.00019986670824589936

# Метод Зейделя

In [237]:
H_R = np.tril(np.ones(n * (n-1) // 2)).T
H_L = np.ones((n, n)) - H_R

In [238]:
H_R = H_R * H_D
H_L = H_L *  H_D

In [239]:
x0 = np.zeros((n, 1))
x1 = x0

CC = np.dot(np.linalg.inv(np.eye(n) - H_L), H_R)
DD = np.linalg.inv(np.eye(n) - H_L)

for i in range(10):
  x0 = x1
  x1 = np.dot(CC, x0) + np.dot(DD, g_D)

In [240]:
x1

array([[0.09999982],
       [0.50000007],
       [0.9999999 ]])

### Фактическая погрешность

In [241]:
fact_ze = np.linalg.norm(x1 - x_g, ord=np.inf)
fact_ze

3.355385834458158e-07

### Спектральный радиус

In [242]:
rho = max(abs(np.linalg.eig(CC)[0]))
rho

0.22062938182072944

# Приближение по Люстернику

In [243]:
x2 = x0 + 1/(1 + rho) * (x1 - x0)

In [244]:
x2

array([[0.09999961],
       [0.50000016],
       [0.99999978]])

In [245]:
fact_lu = np.linalg.norm(x2 - x_g, ord=np.inf)
fact_lu

5.497796264630761e-07

# Метод верхней релаксации

In [246]:
q = 2/(1 + (1 - rho**2)**(1/2))
q

1.0124748459022919

In [247]:

x0 = np.zeros((n,1))
x1 = x1
for t in range(10):
  x0 = x1
  for i in range(n):
    x1[i] = x0[i] + q * (np.dot(H_D[i, :], x1) - x0[i]+ g_D[i])

In [248]:
x1

array([[0.10000016],
       [0.49999994],
       [1.00000009]])

In [249]:
fact_re = np.linalg.norm(x1 - x_g, ord=np.inf)
fact_re

3.862188346914763e-14

# Сравнение методов

### Решение методом Гаусса

In [250]:
x_g

array([[0.10000016],
       [0.49999994],
       [1.00000009]])

### Погрешность метода простой итерации

In [251]:
fact_it

0.00019986670824589936

### Погрешность метода Зейделя

In [252]:
fact_ze

3.355385834458158e-07

### Погрешность метода Зейделя с приближением по Люстернику

In [253]:
fact_lu

5.497796264630761e-07

### Погрешность метода верхней релаксации

In [254]:
fact_re

3.862188346914763e-14